In [3]:
#!pip install nest_asyncio
#!pip install --upgrade telethon

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from telethon import TelegramClient
from datetime import datetime
import nest_asyncio
import pytz
import re

# Enable nested event loops for Jupyter
nest_asyncio.apply()
load_dotenv()

# API credentials
api_id = os.environ.get("TELEGRAM_API_ID")
api_hash = os.environ.get("TELEGRAM_API_HASH")
group_id = -1002041357608
topic_id = 19940
finnish_tz = pytz.timezone('Europe/Helsinki')

# Convert Finnish time to UTC-aware datetime
def convert_to_utc(dt):
    return finnish_tz.localize(dt).astimezone(pytz.utc)

async def get_topic_messages(starttime=None, endtime=None, limit=1000):
    async with TelegramClient('session_name', api_id, api_hash) as client:
        await client.start()
        
        # Convert start/end times to UTC-aware datetimes
        starttime_utc = convert_to_utc(starttime) if starttime else None
        endtime_utc = convert_to_utc(endtime) if endtime else None

        messages = []
        total_fetched = 0

        # We will iterate forward in time starting from starttime_utc
        async for message in client.iter_messages(
            entity=group_id,
            offset_date=starttime_utc,  # Start from around starttime
            reverse=True                # Move forward in time
        ):
            # If message is after endtime, stop
            if endtime_utc and message.date > endtime_utc:
                break
            # If message is before starttime, skip it
            if starttime_utc and message.date < starttime_utc:
                continue

            # Check if this message belongs to the desired topic
            # Note: forum_topic_id may differ based on Telethon version;
            # if not available, print message attributes to find a suitable field.
            if message.reply_to and message.reply_to.reply_to_msg_id == topic_id:
                sender = await message.get_sender()
                sender_name = sender.first_name if sender else "Unknown"
                messages.append({
                    'date': message.date.astimezone(finnish_tz),
                    'sender': sender_name,
                    'text': message.text,
                })

                total_fetched += 1
                if total_fetched >= limit:
                    break

        return messages

        

async def get_messages():
    async with TelegramClient('session_name', api_id, api_hash) as client:
        # Get messages
        messages = await client.get_messages(group_username, limit=10)
        
        # Print messages
        for message in messages:
            print(f'[{message.date}] {message.sender.first_name}: {message.text}')


async def list_dialogs():
    async with TelegramClient('session_name', api_id, api_hash) as client:
        dialogs = await client.get_dialogs()
        for dialog in dialogs:
            print(f"Name: {dialog.name}, ID: {dialog.id}")


# List the Recent dialogs
#await list_dialogs()

# Get the message for the whole group (dialog)
#await get_messages()

# Get the message for a specific topic within the date limit
starttime = datetime(2025, 1, 23, 0, 0, 0) 
endtime = datetime(2025, 1, 30, 0, 0, 0)
messages = await get_topic_messages(starttime=starttime, endtime=endtime)


In [2]:
df = pd.DataFrame(messages)
df

,date,sender,text
0,2025-01-23 02:26:34+02:00,繪圖員 - 娜美,"Sell - Price=0.56643, TP:0.56599, SL:0.56687\n..."
1,2025-01-23 04:15:57+02:00,繪圖員 - 娜美,"Sell - Price=156.517, TP:156.418, SL:156.61599..."
2,2025-01-23 04:16:24+02:00,繪圖員 - 娜美,"Buy - Price=1.23074, TP:1.23159, SL:1.22989\nS..."
3,2025-01-23 04:21:14+02:00,繪圖員 - 娜美,"Buy - Price=1.23082, TP:1.23167, SL:1.22997"
4,2025-01-23 04:26:13+02:00,繪圖員 - 娜美,"Buy - Price=1.23102, TP:1.23187, SL:1.23017"
...,...,...,...
780,2025-01-29 21:45:41+02:00,繪圖員 - 娜美,"Sell - Price=155.318, TP:155.20100000000002, S..."
781,2025-01-29 22:17:09+02:00,繪圖員 - 娜美,"Sell - Price=74.01, TP:73.84, SL:74.18\nSendin..."
782,2025-01-29 22:20:41+02:00,繪圖員 - 娜美,"Sell - Price=2755.74, TP:2753.3199999999997, S..."
783,2025-01-29 22:42:20+02:00,繪圖員 - 娜美,"Sell - Price=73.93, TP:73.76, SL:74.1000000000..."


In [3]:
# Function to extract desired fields using regex
def extract_fields(text):
    # Initialize default values
    type_ = None
    symbol = None
    price = None
    lot = None
    tp = None
    sl = None
    
    # Extract Type: 'Buy' or 'Sell'
    type_match = re.match(r'^(Buy|Sell)', text, re.IGNORECASE)
    if type_match:
        type_ = type_match.group(1).capitalize()
    
    # Extract Symbol: text after 'Sending order ' and before ' Lot:'
    symbol_match = re.search(r'Sending order\s+(\w+)\s+Lot:', text, re.IGNORECASE)
    if symbol_match:
        symbol = symbol_match.group(1)
    
    # Extract Price from the first part (after 'Price=')
    price_match = re.search(r'Price=([\d\.]+)', text)
    if price_match:
        price = float(price_match.group(1))
    
    # Extract Lot: after 'Lot:'
    lot_match = re.search(r'Lot:\s*([\d\.]+)', text, re.IGNORECASE)
    if lot_match:
        lot = float(lot_match.group(1))
    
    # Extract TP: after 'TP:'
    tp_match = re.search(r'TP:\s*([\d\.]+)', text, re.IGNORECASE)
    if tp_match:
        tp = float(tp_match.group(1))
    
    # Extract SL: after 'SL:'
    sl_match = re.search(r'SL:\s*([\d\.]+)', text, re.IGNORECASE)
    if sl_match:
        sl = float(sl_match.group(1))
    
    return pd.Series({
        'Type': type_,
        'Symbol': symbol,
        'Price': price,
        'Lot': lot,
        'TP': tp,
        'SL': sl
    })

# Apply the extraction function to each row in the 'text' column
filtered_df = df[df['text'].str.contains("Sending order")].copy()
filtered_df[['Type', 'Symbol', 'Price', 'Lot', 'TP', 'SL']] = filtered_df['text'].apply(extract_fields)
filtered_df


,date,sender,text,Type,Symbol,Price,Lot,TP,SL
0,2025-01-23 02:26:34+02:00,繪圖員 - 娜美,"Sell - Price=0.56643, TP:0.56599, SL:0.56687\n...",Sell,NZDUSD,0.56643,0.47,0.56599,0.56687
1,2025-01-23 04:15:57+02:00,繪圖員 - 娜美,"Sell - Price=156.517, TP:156.418, SL:156.61599...",Sell,USDJPY,156.51700,0.31,156.41800,156.61600
2,2025-01-23 04:16:24+02:00,繪圖員 - 娜美,"Buy - Price=1.23074, TP:1.23159, SL:1.22989\nS...",Buy,GBPUSD,1.23074,0.23,1.23159,1.22989
8,2025-01-23 05:52:06+02:00,繪圖員 - 娜美,"Sell - Price=102222.6, TP:101810.91, SL:102634...",Sell,BTCUSD,102222.60000,0.05,101810.91000,102634.29000
11,2025-01-23 08:00:42+02:00,繪圖員 - 娜美,"Sell - Price=156.711, TP:156.61800000000002, S...",Sell,USDJPY,156.71100,0.36,156.61800,156.80400
...,...,...,...,...,...,...,...,...,...
771,2025-01-29 21:21:35+02:00,繪圖員 - 娜美,"Buy - Price=0.56447, TP:0.56487, SL:0.56407000...",Buy,NZDUSD,0.56447,3.34,0.56487,0.56407
778,2025-01-29 21:40:58+02:00,繪圖員 - 娜美,"Sell - Price=155.251, TP:155.13400000000001, S...",Sell,USDJPY,155.25100,1.56,155.13400,155.36800
781,2025-01-29 22:17:09+02:00,繪圖員 - 娜美,"Sell - Price=74.01, TP:73.84, SL:74.18\nSendin...",Sell,XTIUSD,74.01000,7.50,73.84000,74.18000
782,2025-01-29 22:20:41+02:00,繪圖員 - 娜美,"Sell - Price=2755.74, TP:2753.3199999999997, S...",Sell,XAUUSD,2755.74000,0.51,2753.32000,2758.16000


In [4]:
filtered_df['datelocal'] = (
    filtered_df['date']
    .dt.tz_convert(finnish_tz)  # Convert to Europe/Helsinki timezone
    .dt.tz_localize(None)        # Remove timezone information
)
filtered_df[['datelocal','Symbol','Type','Price']].to_excel('maptrade/maptrade_signal.xlsx')